In [1]:
import ROOT
import math
import numpy as np

ROOT.gSystem.Load("libDelphes")

try:
    ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
    ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')
except:
    pass

def delta_phi(phi1, phi2):
    dphi = abs(phi1 - phi2)
    return min(dphi, 2 * math.pi - dphi)

def calculate_delta_phi_jet_met(jet_phi, met_phi): 
    return delta_phi(jet_phi, met_phi)

def analyze_specific_points(root_file_path, cross_section):
    # Create chain of root trees
    chain = ROOT.TChain("Delphes")
    chain.Add(root_file_path)

    # Create object of class ExRootTreeReader
    treeReader = ROOT.ExRootTreeReader(chain)
    numberOfEntries = treeReader.GetEntries()

    # Get pointer to the branch containing Particle information
    branchJet = treeReader.UseBranch("Jet")
    branchMET = treeReader.UseBranch("MissingET")
    branchEvent = treeReader.UseBranch("Weight")

    # Events Counter
    initial_events = events_passed_met_cut = events_passed_pt_eta_cut = events_passed_jets_criteria = events_passed_delta_phi_cut = events_passed_leading_jet_cut = 0
    initial_events_sq = events_passed_met_cut_sq = events_passed_pt_eta_cut_sq = events_passed_jets_criteria_sq = events_passed_delta_phi_cut_sq = events_passed_leading_jet_cut_sq = 0

    #### Initial Cuts 
    pt_cut = 30.0
    max_eta = 2.8  

    # Constants for scaling
    luminosity = 140.0 * 1000  # pb
    # cross_section = 0.385  # pb for gluino at 1 TeV

    # Dictionary to store MET cut values and corresponding event counts
    met_cuts = {200: 0,250: 0,300: 0,350: 0,400: 0,500: 0,600: 0,700: 0,800: 0,900: 0,1000: 0,1100: 0,1200: 0 }

    # Initialize dictionary to store squared sum of event weights for each MET cut value
    met_cuts_sq = {met_cut: 0 for met_cut in met_cuts}

    sum_event_weights = 0
    for i in range(numberOfEntries):
        treeReader.ReadEntry(i)
        sum_event_weights += branchEvent.At(6).Weight

    # Loop over all events
    for entry in range(numberOfEntries):
        # Load selected branches with data from specified event
        treeReader.ReadEntry(entry)

        num_jets_meeting_criteria = leading_jet_pt = leading_jet_eta = 0

        initial_events += branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights
        initial_events_sq += (branchEvent.At(6).Weight * luminosity * cross_section / sum_event_weights) ** 2

        if branchMET.At(0).MET > 200:
            events_passed_met_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
            events_passed_met_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # Loop over all jets
            for jet_index in range(branchJet.GetEntries()):
                jet = branchJet.At(jet_index)

                # Jet Pt and Eta Cuts
                if jet.PT > pt_cut and abs(jet.Eta) < max_eta:
                    num_jets_meeting_criteria += 1

                # Save Leading Jet Pt and Eta
                if jet.PT > leading_jet_pt:
                    leading_jet_pt = jet.PT
                    leading_jet_eta = jet.Eta

            # Count events passing pt, eta cut
            if num_jets_meeting_criteria > 0:
                events_passed_pt_eta_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_pt_eta_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

            # To set limit over jets
            if num_jets_meeting_criteria <= 4:
                events_passed_jets_criteria += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                events_passed_jets_criteria_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                # Delta_Phi cut implementation
                delta_phi_jet_met_min = 0.6 if 200 < branchMET.At(0).MET <= 250 else 0.4
                delta_phis = []
                for jet_index in range(branchJet.GetEntries()):
                    jet = branchJet.At(jet_index)
                    delta_phi_jet_met = calculate_delta_phi_jet_met(jet.Phi, branchMET.At(0).Phi)
                    delta_phis.append(delta_phi_jet_met)

                # Check how many jets pass the delta_phi cut
                if all(delta_phi > delta_phi_jet_met_min for delta_phi in delta_phis):
                    events_passed_delta_phi_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                    events_passed_delta_phi_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                    # Leading Jets Pt and Eta cuts
                    if leading_jet_pt > 150 and abs(leading_jet_eta) < 2.4:
                        events_passed_leading_jet_cut += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                        events_passed_leading_jet_cut_sq += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2

                        # Loop over MET cut values
                        for met_cut in met_cuts:
                            if branchMET.At(0).MET > met_cut:
                                met_cuts[met_cut] += (branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights
                                met_cuts_sq[met_cut] += ((branchEvent.At(6).Weight * luminosity * cross_section) / sum_event_weights) ** 2
                                

    print(f"\nAnalysis results for root file: {root_file_path}")
    print("===================================================")

    # Calculate statistical uncertainties for each cut
    initial_events_uncer = np.sqrt(initial_events_sq)
    events_passed_met_cut_uncer = np.sqrt(events_passed_met_cut_sq)
    events_passed_pt_eta_cut_uncer = np.sqrt(events_passed_pt_eta_cut_sq)
    events_passed_jets_criteria_uncer = np.sqrt(events_passed_jets_criteria_sq)
    events_passed_delta_phi_cut_uncer = np.sqrt(events_passed_delta_phi_cut_sq)
    events_passed_leading_jet_cut_uncer = np.sqrt(events_passed_leading_jet_cut_sq)

    print(f"\nInitial number of events: {initial_events:.2f} +/- {initial_events_uncer:.2f} ")
    print(f"\nNumber of events passed MET > 200 GeV cut: {events_passed_met_cut:.2f} +/- {events_passed_met_cut_uncer:.2f}")
    print(f"\nNumber of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: {events_passed_pt_eta_cut:.2f} +/- {events_passed_pt_eta_cut_uncer:.2f}")
    print(f"\nNumber of events passed number of jets <= 4 cut: {events_passed_jets_criteria:.2f} +/- {events_passed_jets_criteria_uncer:.2f}")
    print(f"\nNumber of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: {events_passed_delta_phi_cut:.2f} +/- {events_passed_delta_phi_cut_uncer:.2f}")
    print(f"\nNumber of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: {events_passed_leading_jet_cut:.2f} +/- {events_passed_leading_jet_cut_uncer:.2f}")

    paper_met_cuts = {200: 102274, 250: 41158, 300: 20893, 350: 11937, 400: 7214, 500: 2918,
                      600: 1391, 700: 574, 800: 298, 900: 164, 1000: 186, 1100: 73, 1200: 40}

    ratios = {}  # Dictionary to store the ratios
    ratios_uncer = {} # Dictionary to store the ratios uncertainty
    
    # Calculate the ratio for each MET cut value
    for met_cut in met_cuts:
        if met_cut in paper_met_cuts:
            ratio = met_cuts[met_cut] / paper_met_cuts[met_cut]
            ratio_uncer = np.sqrt(met_cuts_sq[met_cut]) / paper_met_cuts[met_cut]
            ratios[met_cut] = ratio
            ratios_uncer[met_cut] = ratio_uncer
            print(f"\nFor SR MET > {met_cut} GeV: S_Exp: {met_cuts[met_cut]:.2f} +/- {np.sqrt(met_cuts_sq[met_cut]):.2f}, S_Obs: {paper_met_cuts[met_cut]}, and ratio: {ratio:.2f} +/- {ratio_uncer:.2f}")
            

    # Find the MET cut with the maximum ratio
    max_ratio_met_cut = max(ratios, key=ratios.get)
    max_ratio_uncer_met_cut = max(ratios_uncer, key=ratios_uncer.get)

    # Calculate gluino mass point
    gluino_mass_point = cross_section * (paper_met_cuts[max_ratio_met_cut] / met_cuts[max_ratio_met_cut])
    gluino_mass_point_uncer = cross_section * (paper_met_cuts[max_ratio_uncer_met_cut] / met_cuts[max_ratio_uncer_met_cut])

    print("\nThe maximum gluino mass point can be that for which the cross section is around", cross_section, "* (", paper_met_cuts[max_ratio_met_cut], "/", met_cuts[max_ratio_met_cut], ") =", gluino_mass_point, "+/-", gluino_mass_point_uncer)



Welcome to JupyROOT 6.26/06


# For N1 950 ; gl 1000 GeV ; using MLM
#### ### it has entry at 0 for branchEvent.At(6).Weight ; size of 4

In [2]:
# analyze_specific_points("root_files/MLM_N950/tag_1_delphes_events.root", 0.385)

# For N1 850 GeV ; gl 1000 GeV ; using CKKW matching so on

In [3]:
analyze_specific_points("root_files/N850/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N850/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 614.92 

Number of events passed MET > 200 GeV cut: 13352.51 +/- 321.26

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13315.07 +/- 320.81

Number of events passed number of jets <= 4 cut: 11369.84 +/- 295.86

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9067.61 +/- 263.76

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 7924.95 +/- 248.19

For SR MET > 200 GeV: S_Exp: 7924.95 +/- 248.19, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6255.05 +/- 222.52, S_Obs: 41158, and ratio: 0.15 +/- 0.01

For SR MET > 300 GeV: S_Exp: 4744.78 +/- 194.65, S_Obs: 20893, and ratio: 0.23 +/- 0.01

For SR MET > 350 GeV: S_Exp: 3606.94 +/- 170.06, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2735.73 +/- 147.98, S_Obs: 7214, and ratio: 0.38 +/- 0.02

For SR MET > 500 GeV: S_Exp: 16

# For N1 900 GeV ; gl 1000 GeV

In [4]:
analyze_specific_points("root_files/N900/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N900/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 614.92 

Number of events passed MET > 200 GeV cut: 13543.82 +/- 323.19

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13514.66 +/- 322.85

Number of events passed number of jets <= 4 cut: 11611.05 +/- 298.79

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9319.41 +/- 267.24

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8106.51 +/- 250.88

For SR MET > 200 GeV: S_Exp: 8106.51 +/- 250.88, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6305.28 +/- 223.76, S_Obs: 41158, and ratio: 0.15 +/- 0.01

For SR MET > 300 GeV: S_Exp: 4679.32 +/- 193.43, S_Obs: 20893, and ratio: 0.22 +/- 0.01

For SR MET > 350 GeV: S_Exp: 3627.50 +/- 170.68, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2753.52 +/- 148.75, S_Obs: 7214, and ratio: 0.38 +/- 0.02

For SR MET > 500 GeV: S_Exp: 16

# For N1 950 GeV ; gl 1000 GeV 

In [5]:
analyze_specific_points("root_files/N950/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N950/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 614.92 

Number of events passed MET > 200 GeV cut: 13572.21 +/- 323.72

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13534.77 +/- 323.28

Number of events passed number of jets <= 4 cut: 11561.63 +/- 298.20

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9220.30 +/- 266.01

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8087.94 +/- 250.66

For SR MET > 200 GeV: S_Exp: 8087.94 +/- 250.66, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6222.38 +/- 222.18, S_Obs: 41158, and ratio: 0.15 +/- 0.01

For SR MET > 300 GeV: S_Exp: 4693.33 +/- 193.81, S_Obs: 20893, and ratio: 0.22 +/- 0.01

For SR MET > 350 GeV: S_Exp: 3604.89 +/- 169.98, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2672.19 +/- 146.47, S_Obs: 7214, and ratio: 0.37 +/- 0.02

For SR MET > 500 GeV: S_Exp: 16

# For N1 990 GeV ; gl 1000 GeV 

In [6]:
analyze_specific_points("root_files/N990/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N990/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 614.92 

Number of events passed MET > 200 GeV cut: 13513.44 +/- 323.13

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13484.28 +/- 322.79

Number of events passed number of jets <= 4 cut: 11546.86 +/- 298.22

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9240.27 +/- 266.27

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8068.51 +/- 250.42

For SR MET > 200 GeV: S_Exp: 8068.51 +/- 250.42, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6250.29 +/- 222.81, S_Obs: 41158, and ratio: 0.15 +/- 0.01

For SR MET > 300 GeV: S_Exp: 4758.78 +/- 195.03, S_Obs: 20893, and ratio: 0.23 +/- 0.01

For SR MET > 350 GeV: S_Exp: 3586.75 +/- 169.65, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2754.19 +/- 148.52, S_Obs: 7214, and ratio: 0.38 +/- 0.02

For SR MET > 500 GeV: S_Exp: 16

# For N1 990 GeV ; gl 1000 GeV ; again

In [7]:
analyze_specific_points("root_files/N990_again/tag_1_delphes_events.root", 0.385)


Analysis results for root file: root_files/N990_again/tag_1_delphes_events.root

Initial number of events: 53900.00 +/- 614.92 

Number of events passed MET > 200 GeV cut: 13598.09 +/- 323.98

Number of events passed Jet pT > 30 GeV, |eta| < 2.8 cut: 13575.67 +/- 323.71

Number of events passed number of jets <= 4 cut: 11585.94 +/- 298.44

Number of events passed delta phi > 0.4 (> 0.6 if 200 GeV < MET ≤ 250 GeV) cut: 9364.53 +/- 267.89

Number of events passed leading jet pT > 150 GeV, |eta| < 2.4 cut: 8146.51 +/- 251.55

For SR MET > 200 GeV: S_Exp: 8146.51 +/- 251.55, S_Obs: 102274, and ratio: 0.08 +/- 0.00

For SR MET > 250 GeV: S_Exp: 6328.91 +/- 224.19, S_Obs: 41158, and ratio: 0.15 +/- 0.01

For SR MET > 300 GeV: S_Exp: 4799.54 +/- 195.90, S_Obs: 20893, and ratio: 0.23 +/- 0.01

For SR MET > 350 GeV: S_Exp: 3569.77 +/- 169.24, S_Obs: 11937, and ratio: 0.30 +/- 0.01

For SR MET > 400 GeV: S_Exp: 2739.76 +/- 148.18, S_Obs: 7214, and ratio: 0.38 +/- 0.02

For SR MET > 500 GeV: S_E